In [8]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [42]:
import sys
import os
from math import log
import numpy as np
import scipy as sp
from PIL import Image
import matplotlib.pyplot as plt
import pandas as pd


In [36]:
pe_data_path = "/content/drive/MyDrive/content/drive/MyDrive/progetto_SS/benign_data/benign"
bytes_data_path = "/content/drive/MyDrive/content/drive/MyDrive/progetto_SS/benign_data/benign_byte"
img_data_path = "/content/drive/MyDrive/content/drive/MyDrive/progetto_SS/benign_data/benign_imgs"
csv_data_path = "/content/drive/MyDrive/content/drive/MyDrive/progetto_SS/benign_data/data.csv"

In [22]:

def pe2hex(file_path, output_file_path):
  print('Processing '+file_path)
  file = bytearray(open(file_path, 'rb').read())
  key = "\0"
  with open(output_file_path, 'w') as output:
      for count, byte in enumerate(file, 1):
          output.write(
              f'{byte ^ ord(key[(count - 1) % len(key)]):#0{4}x}' + (
                  '\n' if not count % 16 else ' '))


In [46]:
def hex2img(array, output_img_path):
    if array.shape[1]!=16: #If not hexadecimal
        assert(False)
    b=int((array.shape[0]*16)**(0.5))
    b=2**(int(log(b)/log(2))+1)
    a=int(array.shape[0]*16/b)
    print(a,b,array.shape)
    array=array[:a*b//16,:]
    array=np.reshape(array,(a,b))
    im = Image.fromarray(np.uint8(array))
    #out = im.transpose(Image.FLIP_LEFT_RIGHT )
    im.save(output_img_path, "PNG")
    return im





**Convert PE files to binary code**






In [ ]:
files= os.listdir(pe_data_path)

for counter, name in enumerate(files):
  name_output = name.split(".")[0]
  print(name_output)
  pe2hex(os.path.join(pe_data_path,name), os.path.join(bytes_data_path,name_output+".bytes"))

**Convert binary code to Inmage**

In [44]:
df_benign = pd.read_csv(csv_data_path, index_col=False)
already_transformed = list(df_benign['img_code'])
benign_class_index = 25

In [47]:
files= os.listdir(bytes_data_path)

for counter, name in enumerate(files):
        name_output = name.split(".")[0]

        output_image_path = os.path.join(img_data_path,name_output+".png")
        relative_image_path = os.path.join("./benign_data/benign_imgs", name_output+".png")

        if(relative_image_path in already_transformed):
         continue

        print('Processing '+output_image_path)

        f=open(os.path.join(bytes_data_path,name), 'r')

        array=[]
        for line in f:
            xx=line.replace("\n", "").split(" ")

            if(len(xx) != 16 or "" in xx):
              continue

            array.append([int(i,16) if i!='??' else 0 for i in xx])

        img = hex2img(np.array(array),output_image_path)
        del array

        df_benign.loc[len(df_benign.index)] = [relative_image_path, benign_class_index]

        f.close()

Processing /content/drive/MyDrive/content/drive/MyDrive/progetto_SS/benign_data/benign_imgs/1093d64194b6d6ad61185f5996ffa0b474dfdc88f0eb793479573fe1d7550929.png
2188 4096 (560166, 16)
Processing /content/drive/MyDrive/content/drive/MyDrive/progetto_SS/benign_data/benign_imgs/0f4ecc7f67b6d098ba1dae6e2b9700caeb7a736054e6de1286835023d328decb.png
524 2048 (67158, 16)
Processing /content/drive/MyDrive/content/drive/MyDrive/progetto_SS/benign_data/benign_imgs/05bff93d5b773dd409d8677c4601d76a774a8ecaf6f96e8a6a06a40daee4053c.png
214 256 (3424, 16)
Processing /content/drive/MyDrive/content/drive/MyDrive/progetto_SS/benign_data/benign_imgs/06ee87f1ff7d24764b66261adb6acfd0c378b62fb3a4403349a5861e06bc540b.png
122 128 (981, 16)
Processing /content/drive/MyDrive/content/drive/MyDrive/progetto_SS/benign_data/benign_imgs/14560f2199a38d754dd865b162d8c50ea50f0da5514b763e1e7bfb0f06f4a35b.png
100 128 (806, 16)
Processing /content/drive/MyDrive/content/drive/MyDrive/progetto_SS/benign_data/benign_imgs/16f3

In [48]:
df_benign.to_csv(csv_data_path, index=False)